# Oļegs Korsaks, 051RDB146

In [1]:
from math import exp
from operator import methodcaller

import numpy as np
import tensorflow as tf

from sklearn.datasets import load_iris

In [2]:
letters = [
    ['A',[0,1,0,1,0,1,1,1,1,1,0,1,1,0,1],[1,0,0]],
    ['E',[1,1,1,1,0,0,1,1,0,1,0,0,1,1,1],[1,0,0]],
    ['D',[1,1,0,1,0,1,1,0,1,1,0,1,1,1,0],[1,0,0]],
    ['B',[1,1,0,1,0,1,1,1,0,1,0,1,1,1,0],[0,1,0]],
    ['C',[1,1,1,1,0,0,1,0,0,1,0,0,1,1,1],[0,1,0]],
    ['F',[1,1,1,1,0,0,1,1,0,1,0,0,1,0,0],[0,1,0]],
    ['G',[1,1,1,1,0,0,1,0,1,1,0,1,1,1,1],[0,0,1]],
    ['H',[1,0,1,1,0,1,1,1,1,1,0,1,1,0,1],[0,0,1]],
    ['I',[0,1,0,0,1,0,0,1,0,0,1,0,0,1,0],[0,0,1]],
]

In [3]:
def a_step(val):
    return int(val>0)


def a_lin(val):
    if val <= 0:
        return 0
    elif val > 5:
        return 1
    
    return val/5


def a_sigmoid(val):
    return 1.0 / (1.0+exp(-val))


def a_tanh(val):
    return (exp(val)-exp(-val)) / (exp(val)+exp(-val))

In [4]:
learning_rate = 0.1


class Neiron:
    def __init__(self, in_size):
        self.weights = np.zeros(in_size)
        self.w0 = 0

    def classify(self, incoming_signals):
        net = np.dot(incoming_signals, self.weights) + self.w0
        
        return a_step(net)
    
    def learn(self, incoming_signals, c):
        y = self.classify(incoming_signals)
        delta = c - y
        
        if delta:
            for i in range(len(self.weights)):
                self.weights[i] += delta * learning_rate * incoming_signals[i]

            self.w0 += delta * learning_rate

In [5]:
def run_neirons(data: list, neiron_class, in_size: int, class_count: int, epoch_count: int = 100):
    neirons = tuple(neiron_class(in_size) for _ in range(class_count))
    
    for epoch in range(epoch_count):
        for sample in data:
            for i, neiron in enumerate(neirons):
                neiron.learn(sample[1], sample[2][i])
    
    for sample in data:
        method = methodcaller('classify', sample[1])
        print(f'{sample[0]} {tuple(map(method, neirons))}')
    
    for neiron in neirons:
        print(neiron.weights)

In [6]:
run_neirons(letters, Neiron, 15, 3, 100)

A (1, 0, 0)
E (1, 0, 0)
D (1, 0, 0)
B (0, 1, 0)
C (0, 1, 0)
F (0, 1, 0)
G (0, 0, 1)
H (0, 0, 1)
I (0, 0, 1)
[-0.7  0.5 -0.8  0.  -0.6 -0.3  0.   0.9  1.8  0.  -0.6 -1.   0.   0.3
  0.4]
[ 0.2  0.2 -0.4  0.2 -0.2  0.4  0.2 -0.7 -1.7  0.2 -0.2 -0.1  0.2  0.2
 -0.4]
[ 0.2 -0.3  0.3 -0.2  0.3 -0.1 -0.2 -0.2  0.4 -0.2  0.3  0.4 -0.2  0.1
 -0.1]


In [7]:
iris = load_iris()
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(4, kernel_regularizer=tf.keras.regularizers.l2(0.1), activation='sigmoid', input_shape=(4, )))
model.add(tf.keras.layers.Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy')

targets = np.zeros((len(iris.target), 3))

for ti, iris_target in enumerate(iris.target):
    targets[ti][iris_target] = 1

model.fit(iris.data, targets, epochs=1000, verbose=0)
predictions = model.predict(iris.data)

print(predictions)

[[0.9204139  0.06755938 0.01202674]
 [0.915459   0.07146218 0.01307879]
 [0.9275438  0.06192729 0.0105289 ]
 [0.91499895 0.071821   0.01318   ]
 [0.92334116 0.06524835 0.01141051]
 [0.8930644  0.08869142 0.01824423]
 [0.9232385  0.065335   0.01142646]
 [0.91368014 0.07284008 0.01347968]
 [0.92136914 0.06682571 0.0118051 ]
 [0.9133447  0.07311273 0.0135426 ]
 [0.91233665 0.07387634 0.01378708]
 [0.9098151  0.07584978 0.0143352 ]
 [0.91994435 0.0679449  0.0121107 ]
 [0.9418108  0.0504672  0.00772205]
 [0.9298346  0.06008018 0.01008524]
 [0.9131037  0.07325332 0.01364296]
 [0.9227762  0.06568228 0.01154143]
 [0.917497   0.06984968 0.01265337]
 [0.88866806 0.09202816 0.01930379]
 [0.91568035 0.07126424 0.01305551]
 [0.8902302  0.09086621 0.01890358]
 [0.9108166  0.07505736 0.01412607]
 [0.9471329  0.04611335 0.00675379]
 [0.88154876 0.09741215 0.02103906]
 [0.88436043 0.09530515 0.02033443]
 [0.8985608  0.08453838 0.01690084]
 [0.8991487  0.08407004 0.01678129]
 [0.91215205 0.07402716 0.01

## Mājasdarbs 1

Pārveidosim neirona klasi, lai viņš varētu klasificēt 4 klašu simbolus. Pievienosim ! _ un . simbolus

In [8]:
letters = [
    ['A',[0,1,0,1,0,1,1,1,1,1,0,1,1,0,1],[1,0,0,0]],
    ['E',[1,1,1,1,0,0,1,1,0,1,0,0,1,1,1],[1,0,0,0]],
    ['D',[1,1,0,1,0,1,1,0,1,1,0,1,1,1,0],[1,0,0,0]],
    ['B',[1,1,0,1,0,1,1,1,0,1,0,1,1,1,0],[0,1,0,0]],
    ['C',[1,1,1,1,0,0,1,0,0,1,0,0,1,1,1],[0,1,0,0]],
    ['F',[1,1,1,1,0,0,1,1,0,1,0,0,1,0,0],[0,1,0,0]],
    ['G',[1,1,1,1,0,0,1,0,1,1,0,1,1,1,1],[0,0,1,0]],
    ['H',[1,0,1,1,0,1,1,1,1,1,0,1,1,0,1],[0,0,1,0]],
    ['I',[0,1,0,0,1,0,0,1,0,0,1,0,0,1,0],[0,0,1,0]],
    
    ['!',[0,1,0,0,1,0,0,1,0,0,0,0,0,1,0],[0,0,0,1]],
    ['_',[0,0,0,0,0,0,0,0,0,0,0,0,1,1,1],[0,0,0,1]],
    ['.',[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],[0,0,0,1]],
]


class NeironHome:
    def __init__(self, in_size):
        self.weights = np.zeros(in_size)
        self.w0 = 0

    def classify(self, incoming_signals):
        net = np.dot(incoming_signals, self.weights) + self.w0
        
        return a_sigmoid(net)
    
    def learn(self, incoming_signals, c):
        y = self.classify(incoming_signals)
        delta = c - y
        
        if delta:
            for i in range(len(self.weights)):
                self.weights[i] += delta * learning_rate * incoming_signals[i]

            self.w0 += delta * learning_rate

In [9]:
run_neirons(letters, NeironHome, 15, 4, 1000)

A (0.9975535051097201, 6.660629571448497e-06, 0.008248493415553408, 0.00013232113048282796)
E (0.8293688905071558, 0.07107663617372244, 0.00345475745807826, 0.0008908342725369956)
D (0.8834289329308431, 0.040197189905192325, 0.005393751338105291, 0.000374360388021805)
B (0.06296538404255056, 0.9611077227614221, 2.5487150667717718e-05, 0.00044414160389372084)
C (0.08063124850588929, 0.941473518306242, 0.00588321544731504, 0.0016078235509806357)
F (0.02633476838322883, 0.9911138221082045, 0.0034084107149066696, 2.9723027415453927e-05)
G (0.04066958299013252, 0.00015968439826207983, 0.9849054420714136, 0.0002711874602893066)
H (0.02117992296299061, 1.3000067100018376e-05, 0.9923537292619337, 1.8435006117880236e-05)
I (0.0037253089873815924, 2.5566882336613036e-05, 0.9800763983367607, 0.02047026562186848)
! (0.018415486080804077, 4.3871158090628676e-05, 0.013755496597630701, 0.9781077632338467)
_ (0.00838609151522248, 0.00038552142987693667, 0.0020254630608827185, 0.9968259609187382)
. (0.

**Ir redzams, ka "!", "\_" un "." simboli tiek klasificēti kā 4. klase. Citu burtu klasifikācija nemainās.**

Pārtaisīsim TensorFlow kodu, lai viņš klasificētu burtus:

In [13]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(15, kernel_regularizer=tf.keras.regularizers.l2(0.1), activation='sigmoid', input_shape=(15, )))
model.add(tf.keras.layers.Dense(4, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy')

np_letters = np.array(tuple(np.array(letter[1]) for letter in letters))
print(np_letters)
targets = np.array(tuple(np.array(letter[2]) for letter in letters))
print(targets)

model.fit(np_letters, targets, epochs=1000, verbose=0)
predictions = model.predict(np_letters)

print(predictions)

[[0 1 0 1 0 1 1 1 1 1 0 1 1 0 1]
 [1 1 1 1 0 0 1 1 0 1 0 0 1 1 1]
 [1 1 0 1 0 1 1 0 1 1 0 1 1 1 0]
 [1 1 0 1 0 1 1 1 0 1 0 1 1 1 0]
 [1 1 1 1 0 0 1 0 0 1 0 0 1 1 1]
 [1 1 1 1 0 0 1 1 0 1 0 0 1 0 0]
 [1 1 1 1 0 0 1 0 1 1 0 1 1 1 1]
 [1 0 1 1 0 1 1 1 1 1 0 1 1 0 1]
 [0 1 0 0 1 0 0 1 0 0 1 0 0 1 0]
 [0 1 0 0 1 0 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]]
[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]]
[[0.40122816 0.27044576 0.27229726 0.05602879]
 [0.3218913  0.33545253 0.24915652 0.09349957]
 [0.3813687  0.30198744 0.25019044 0.0664534 ]
 [0.38808408 0.33379495 0.21402305 0.06409793]
 [0.28799808 0.33435825 0.2553881  0.12225551]
 [0.32503438 0.35284585 0.23231284 0.08980686]
 [0.33067662 0.28039443 0.31060755 0.07832137]
 [0.3395056  0.2619369  0.33114058 0.06741691]
 [0.0762201  0.15722476 0.28797558 0.47857964]
 [0.0646231  0.14966469 0.24060789 0.5451044 ]


**Interesanti**, bet TensorFlow versija apstiprināja manas šaubas par "I" un "!" lidzīguma problēmu, kāmēr bez TensorFlow klasifikācija nostrādāja pareizi. Ar parējiem burtiem arī ir redzamas problēmas - B tiek pieskaitīts pie 2. klases. Pārbaudīšu, vai epohu skaits uzlabos situāciju:

In [14]:
model.fit(np_letters, targets, epochs=10000, verbose=0)
predictions = model.predict(np_letters)

print(predictions)

[[9.5445937e-01 4.3652891e-03 4.0330362e-02 8.4496563e-04]
 [7.2989631e-01 2.2257757e-01 4.1493177e-02 6.0328790e-03]
 [8.2664573e-01 1.2553719e-01 4.2144276e-02 5.6728711e-03]
 [1.4880331e-01 8.4769565e-01 6.0119445e-04 2.8998198e-03]
 [1.5680043e-01 8.0118918e-01 2.2088222e-02 1.9922161e-02]
 [4.3666609e-02 9.4789302e-01 2.4487793e-03 5.9915907e-03]
 [8.4991813e-02 1.3039259e-03 9.1236556e-01 1.3388292e-03]
 [5.0192457e-02 8.0565619e-04 9.4769442e-01 1.3074677e-03]
 [1.1133432e-02 1.4172719e-03 8.9987338e-01 8.7575883e-02]
 [8.3672600e-03 1.4911260e-02 7.1887158e-02 9.0483433e-01]
 [8.3632040e-04 1.0093537e-02 1.6791027e-02 9.7227907e-01]
 [7.0124064e-05 6.3012759e-03 4.6660425e-03 9.8896259e-01]]
